In [1]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging
print(xgb.__version__)




1.6.0-dev


In [2]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM-80GB (UUID: GPU-a5bb48d9-8574-a189-5dcf-39fe0abaef3e)
GPU 1: NVIDIA A100-SXM-80GB (UUID: GPU-3e39a349-3198-0ac9-0443-0e1394522a48)
GPU 2: NVIDIA A100-SXM-80GB (UUID: GPU-d3621334-6770-a11d-92a9-b1ccf5341afa)
GPU 3: NVIDIA DGX Display (UUID: GPU-6b72d40d-8be4-ff90-6b1b-4ff7f11c759b)
GPU 4: NVIDIA A100-SXM-80GB (UUID: GPU-b5b5d840-898c-26f8-c05b-abe0640b381f)


In [3]:
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=['GPU-a5bb48d9-8574-a189-5dcf-39fe0abaef3e','GPU-3e39a349-3198-0ac9-0443-0e1394522a48',
                                                'GPU-d3621334-6770-a11d-92a9-b1ccf5341afa', 'GPU-b5b5d840-898c-26f8-c05b-abe0640b381f'])
client = Client(cluster)

distributed.diskutils - INFO - Found stale lock file and directory '/home/btunguz/Programming/Kaggle/Home_Credit/scripts/dask-worker-space/worker-jexf1hpy', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/btunguz/Programming/Kaggle/Home_Credit/scripts/dask-worker-space/worker-kdptv462', purging
distributed.diskutils - INFO - Found stale lock file and directory '/home/btunguz/Programming/Kaggle/Home_Credit/scripts/dask-worker-space/worker-whfwtxdv', purging
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


In [4]:
!nvidia-smi

Wed Dec  8 07:23:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   36C    P0    65W / 275W |    414MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:47:00.0 Off |                    0 |
| N/A   

In [5]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold = dd.read_csv(f'../input/xgtrain_fold_{i}.csv')
    val_fold = dd.read_csv(f'../input/xgval_fold_{i}.csv')
    
    train_fold = train_fold.replace([np.inf, -np.inf], np.nan)
    val_fold = val_fold.replace([np.inf, -np.inf], np.nan)
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)
    



Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [6]:
application_train = pd.read_csv('../input/application_train.csv')
target = application_train.TARGET.values
train_oof = np.zeros((application_train.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']), 
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_categorical('max_depth', [3,5,7,9,11,13,15,17,20]),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }
    
    #start_time = time()
    kf = StratifiedKFold(5, shuffle=True, random_state=1974)

    for i, (train_index, val_index) in enumerate(kf.split(application_train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i], train_ys[i])
        dval = xgb.dask.DaskDMatrix(client, val_folds[i], val_ys[i])
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()


    acc = roc_auc_score(target, train_oof)
    
    return acc



In [7]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_4.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.load_study(storage="sqlite:///xgb_optuna_home_credit.db", study_name="five_fold_optuna_xgb_3")


In [8]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=1)


[07:25:49] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 0
[07:25:49] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 1
[07:25:49] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 2
[07:25:49] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 3
[07:27:50] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 0
[07:27:50] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 1
[07:27:50] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 2
[07:27:50] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 3
[07:29:52] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 0
[07:29:52] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 1
[07:29:52] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 2
[07:29:52] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 3
[07:31:50] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 0
[07:31:50] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 1
[07:31

CPU times: user 29.2 s, sys: 5.74 s, total: 34.9 s
Wall time: 10min 3s


In [9]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.790110,0.009803,0.969189,logloss,0.002616,0.089619,13,281,binary:logistic,0.754019,gpu_hist,COMPLETE
1,1,0.790003,0.370391,0.449185,logloss,0.051186,0.015264,3,190,binary:logistic,0.477814,gpu_hist,COMPLETE
2,2,0.773821,1.813089,0.761205,logloss,0.054161,0.001571,11,229,binary:logistic,0.953142,gpu_hist,COMPLETE
3,3,0.783346,0.341957,0.865510,logloss,0.052044,0.003738,13,156,binary:logistic,0.729182,gpu_hist,COMPLETE


In [10]:
df.to_csv('optuna_xgb_output_3.csv', index=False)

In [11]:
%%time
study.optimize(objective, n_trials=3)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_3.csv', index=False)
df.head()

[07:37:21] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 0
[07:37:21] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 1
[07:37:21] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 2
[07:37:21] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 3
[07:40:30] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 0
[07:40:30] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 1
[07:40:30] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 2
[07:40:30] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 3
[07:43:46] task [xgboost.dask]:tcp://10.110.42.38:33517 got new rank 0
[07:43:46] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 1
[07:43:46] task [xgboost.dask]:tcp://10.110.42.38:34679 got new rank 2
[07:43:46] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 3
[07:46:57] task [xgboost.dask]:tcp://10.110.42.38:42747 got new rank 0
[07:46:57] task [xgboost.dask]:tcp://10.110.42.38:35411 got new rank 1
[07:46

CPU times: user 1min 15s, sys: 24.3 s, total: 1min 39s
Wall time: 27min 20s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.790110,0.009803,0.969189,logloss,0.002616,0.089619,13,281,binary:logistic,0.754019,gpu_hist,COMPLETE
1,1,0.790003,0.370391,0.449185,logloss,0.051186,0.015264,3,190,binary:logistic,0.477814,gpu_hist,COMPLETE
2,2,0.773821,1.813089,0.761205,logloss,0.054161,0.001571,11,229,binary:logistic,0.953142,gpu_hist,COMPLETE
3,3,0.783346,0.341957,0.865510,logloss,0.052044,0.003738,13,156,binary:logistic,0.729182,gpu_hist,COMPLETE
4,4,0.786806,0.163520,0.774654,logloss,0.001761,0.003448,20,71,binary:logistic,0.800843,gpu_hist,COMPLETE
